In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pwd

/content


In [4]:
%cd drive/My Drive/prepro

/content/drive/My Drive/prepro


In [5]:
!ls

calendar.csv  model20.h5  model6.h5	preprocessing.ipynb
dataset.csv   model21.h5  model7.h5	prepro_data.csv
model10.h5    model22.h5  model8.h5	sales_train_evaluation.csv
model15.h5    model23.h5  model9_C0.h5	sample_submission.csv
model16.h5    model2.h5   model9_C1.h5	second_pred.csv
model17.h5    model3.h5   model9_C2.h5	sell_prices.csv
model18.h5    model4.h5   model.h5	tcn_train.ipynb
model19.h5    model5.h5   new_data.csv	y_test.csv


In [6]:
import torch
import torch.optim as optim
import numpy as np
import os
import pandas as pd

In [7]:
from tqdm import tqdm
from sklearn.preprocessing import OrdinalEncoder

In [8]:
df = pd.read_csv("prepro_data.csv")
df.info()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5918109 entries, 0 to 5918108
Data columns (total 34 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   Unnamed: 0.1       int64  
 2   id                 object 
 3   item_id            int64  
 4   dept_id            int64  
 5   cat_id             int64  
 6   store_id           int64  
 7   state_id           int64  
 8   day                int64  
 9   demand             int64  
 10  wm_yr_wk           int64  
 11  weekday            object 
 12  wday               int64  
 13  month              int64  
 14  year               int64  
 15  d                  object 
 16  event_name_1       object 
 17  event_type_1       object 
 18  event_name_2       object 
 19  event_type_2       object 
 20  snap_CA            int64  
 21  snap_TX            int64  
 22  snap_WI            int64  
 23  sell_price         float64
 24  week_day           int64  
 25  lag_t28           

In [9]:
pip install keras-tcn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [10]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, concatenate, Flatten

from tcn import TCN
from sklearn.preprocessing import MinMaxScaler

In [11]:
#Train set 설정
train = df[df.dn < 1886]
train.fillna(0)

,Unnamed: 0,Unnamed: 0.1,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,...,week_day,lag_t28,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_std_t7,rolling_std_t30,dn
0,0,0,HOBBIES_1_001_CA_1_evaluation,1437,3,1,0,0,29,0,...,5,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1
1,1,1,HOBBIES_1_002_CA_1_evaluation,1438,3,1,0,0,29,0,...,5,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1
2,2,2,HOBBIES_1_003_CA_1_evaluation,1439,3,1,0,0,29,0,...,5,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1
3,3,3,HOBBIES_1_004_CA_1_evaluation,1440,3,1,0,0,29,0,...,5,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1
4,4,4,HOBBIES_1_005_CA_1_evaluation,1441,3,1,0,0,29,0,...,5,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5747360,5747360,5747360,FOODS_3_823_CA_1_evaluation,1432,2,0,0,0,27,0,...,6,8.0,2.285,2.500,3.550,3.445,2.867,3.592,3.070,1885
5747361,5747361,5747361,FOODS_3_824_CA_1_evaluation,1433,2,0,0,0,27,0,...,6,0.0,1.286,2.467,3.550,3.434,2.867,2.984,3.094,1885
5747362,5747362,5747362,FOODS_3_825_CA_1_evaluation,1434,2,0,0,0,27,1,...,6,2.0,1.571,2.200,3.584,3.445,2.877,2.936,2.746,1885
5747363,5747363,5747363,FOODS_3_826_CA_1_evaluation,1435,2,0,0,0,27,0,...,6,0.0,1.571,2.033,3.584,3.445,2.877,2.936,2.723,1885


In [12]:
#Validation set 설정
validation = df[(df.dn >= 1886) & (df.dn < 1914)]
validation.fillna(0)

,Unnamed: 0,Unnamed: 0.1,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,...,week_day,lag_t28,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_std_t7,rolling_std_t30,dn
5747365,5747365,5747365,HOBBIES_1_001_CA_1_evaluation,1437,3,1,0,0,28,1,...,0,2.0,1.7140,1.866,3.232,3.455,2.873,2.928,2.635,1886
5747366,5747366,5747366,HOBBIES_1_002_CA_1_evaluation,1438,3,1,0,0,28,1,...,0,1.0,1.8570,1.767,2.867,3.467,2.861,2.854,2.610,1886
5747367,5747367,5747367,HOBBIES_1_003_CA_1_evaluation,1439,3,1,0,0,28,0,...,0,0.0,0.7144,1.566,2.834,3.467,2.861,0.951,2.500,1886
5747368,5747368,5747368,HOBBIES_1_004_CA_1_evaluation,1440,3,1,0,0,28,0,...,0,0.0,0.7144,1.566,2.800,3.455,2.855,0.951,2.500,1886
5747369,5747369,5747369,HOBBIES_1_005_CA_1_evaluation,1441,3,1,0,0,28,1,...,0,5.0,1.1430,1.667,2.684,3.500,2.867,1.864,2.578,1886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5832732,5832732,5832732,FOODS_3_823_CA_1_evaluation,1432,2,0,0,0,24,1,...,6,7.0,4.4300,4.066,4.832,4.277,3.861,2.820,5.710,1913
5832733,5832733,5832733,FOODS_3_824_CA_1_evaluation,1433,2,0,0,0,24,0,...,6,1.0,3.5720,4.066,4.850,4.266,3.861,2.820,5.710,1913
5832734,5832734,5832734,FOODS_3_825_CA_1_evaluation,1434,2,0,0,0,24,2,...,6,2.0,3.7150,4.133,4.867,4.254,3.873,2.690,5.670,1913
5832735,5832735,5832735,FOODS_3_826_CA_1_evaluation,1435,2,0,0,0,24,0,...,6,0.0,3.4280,4.066,4.867,4.223,3.867,2.992,5.710,1913


In [13]:
#Test set 설정
test = df[(df.dn >= 1914)]
test.fillna(0)

,Unnamed: 0,Unnamed: 0.1,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,...,week_day,lag_t28,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_std_t7,rolling_std_t30,dn
5832737,5832737,5832737,HOBBIES_1_001_CA_1_evaluation,1437,3,1,0,0,25,0,...,0,1.0,2.000,4.000,4.617,4.180,3.840,2.450,5.734,1914
5832738,5832738,5832738,HOBBIES_1_002_CA_1_evaluation,1438,3,1,0,0,25,0,...,0,2.0,1.857,3.934,4.250,4.188,3.810,2.410,5.746,1914
5832739,5832739,5832739,HOBBIES_1_003_CA_1_evaluation,1439,3,1,0,0,25,0,...,0,0.0,0.857,3.633,4.215,4.180,3.805,0.900,5.707,1914
5832740,5832740,5832740,HOBBIES_1_004_CA_1_evaluation,1440,3,1,0,0,25,0,...,0,11.0,2.285,3.934,4.383,4.300,3.867,3.945,5.855,1914
5832741,5832741,5832741,HOBBIES_1_005_CA_1_evaluation,1441,3,1,0,0,25,1,...,0,2.0,2.285,4.000,4.184,4.320,3.873,3.945,5.820,1914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5918104,5918104,5918104,FOODS_3_823_CA_1_evaluation,1432,2,0,0,0,22,2,...,6,4.0,3.715,4.168,4.883,4.656,4.110,2.690,4.580,1941
5918105,5918105,5918105,FOODS_3_824_CA_1_evaluation,1433,2,0,0,0,22,0,...,6,0.0,2.572,4.133,4.883,4.566,4.105,2.225,4.605,1941
5918106,5918106,5918106,FOODS_3_825_CA_1_evaluation,1434,2,0,0,0,22,1,...,6,2.0,2.857,4.200,4.918,4.590,4.117,1.952,4.560,1941
5918107,5918107,5918107,FOODS_3_826_CA_1_evaluation,1435,2,0,0,0,22,1,...,6,0.0,2.572,3.900,4.900,4.555,4.110,2.225,4.527,1941


In [ ]:
del df

In [ ]:
X_train = train[["month", "day", "wday", "store_id", "cat_id", "dept_id", "item_id", "snap_CA"]]
y_train = train[["demand"]]

In [ ]:
X_val = validation[["month", "day", "wday", "store_id", "cat_id", "dept_id", "item_id", "snap_CA"]]
y_val = validation[["demand"]]

In [ ]:
X_test = test[["month", "day", "wday", "store_id", "cat_id", "dept_id", "item_id","snap_CA"]]
y_test = test[["demand"]]

In [ ]:
print(X_train)

         month  day  wday  store_id  cat_id  dept_id  item_id  snap_CA
0            1   29     1         0       1        3     1437        0
1            1   29     1         0       1        3     1438        0
2            1   29     1         0       1        3     1439        0
3            1   29     1         0       1        3     1440        0
4            1   29     1         0       1        3     1441        0
...        ...  ...   ...       ...     ...      ...      ...      ...
5747360      3   27     2         0       0        2     1432        0
5747361      3   27     2         0       0        2     1433        0
5747362      3   27     2         0       0        2     1434        0
5747363      3   27     2         0       0        2     1435        0
5747364      3   27     2         0       0        2     1436        0

[5747365 rows x 8 columns]


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5747365 entries, 0 to 5747364
Data columns (total 8 columns):
 #   Column    Dtype
---  ------    -----
 0   month     int64
 1   day       int64
 2   wday      int64
 3   store_id  int64
 4   cat_id    int64
 5   dept_id   int64
 6   item_id   int64
 7   snap_CA   int64
dtypes: int64(8)
memory usage: 394.6 MB


In [ ]:
from keras.models import Model as KerasModel
from keras.layers import Activation, Reshape, LSTM
from keras.layers import Concatenate
from keras import optimizers
from keras.callbacks import EarlyStopping
import keras.backend as K
from tcn import tcn_full_summary

In [ ]:
X = np.array(X_train)
y = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
def split_features(X):
    X_list = []
    for i in range(8):
        X_list.append(X[:,i])
    
    return X_list

In [ ]:
class TCNmodel():
    
    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        self._build_model()
        self.fit(X_train, y_train, X_val, y_val)
        
    def preprocess(self, X):
        X_list = split_features(X)
        return X_list
        
    def _build_model(self):

        inp_month = Input(shape=(1,))
        out_month = Embedding(12+1, 7, name='month_embedding')(inp_month)
        out_month = Reshape(target_shape=(7,))(out_month)
        
        inp_day = Input(shape=(1,))
        out_day = Embedding(31+1, 16, name='day_embedding')(inp_day)
        out_day = Reshape(target_shape=(16,))(out_day)
        
        inp_weekday = Input(shape=(1,))
        out_weekday = Embedding(7+1, 4, name='weekday_embedding')(inp_weekday)
        out_weekday = Reshape(target_shape=(4,))(out_weekday)
        
        inp_stores = Input(shape=(1,))
        out_stores = Embedding(10+1, 6, name='stores_embedding')(inp_stores)
        out_stores = Reshape(target_shape=(6,))(out_stores)

        inp_ctgos = Input(shape=(1,))
        out_ctgos = Embedding(3+1, 2, name='categori_embedding')(inp_ctgos)
        out_ctgos = Reshape(target_shape=(2,))(out_ctgos)

        inp_depts = Input(shape=(1,))
        out_depts = Embedding(7+1, 4, name='department_embedding')(inp_depts)
        out_depts = Reshape(target_shape=(4,))(out_depts)
        
        inp_items = Input(shape=(1,))
        out_items = Embedding(3049+1, 50, name='items_embedding')(inp_items)
        out_items = Reshape(target_shape=(50,))(out_items)
        
        inp_snapCA = Input(shape=(1,))
        out_snapCA = Reshape(target_shape=(1,))(inp_snapCA)

        inp_model = [inp_month, inp_day, inp_weekday, inp_stores, inp_ctgos, inp_depts, inp_items, inp_snapCA]
        out_embeddings = [out_month, out_day, out_weekday, out_stores, out_ctgos, out_depts, out_items, out_snapCA]
        
        out_model = Concatenate()(out_embeddings)
        out_model= Reshape(target_shape=(90,1))(out_model)
        out_model= TCN(142,kernel_size=2,return_sequences=False)(out_model)
        out_model= Reshape(target_shape=(142,1))(out_model)
        out_model= TCN(96,kernel_size=2,use_skip_connections=True,return_sequences=False)(out_model)
        out_model = Dense(64)(out_model)
        out_model = Activation('relu')(out_model)
        out_model = Dense(32)(out_model)
        out_model = Activation('relu')(out_model)
        out_model = Dense(1)(out_model)
        
        self.model = KerasModel(inputs=inp_model, outputs=out_model)
        
        self.model.compile(optimizer='Adam', metrics=['mean_absolute_error'],loss=[keras.losses.mean_squared_error])
        
    
    def fit(self, X_train, y_train, X_val, y_val):
        self.model.fit(self.preprocess(X_val), y_val,
                       validation_data=(self.preprocess(X_val), y_val),
                       epochs = 3, batch_size = 1000,
                   )
    
    def prediction(self, X):
        return self.model.predict(self.preprocess(X)).flatten()

In [ ]:
model = TCNmodel(X, y, X_val, y_val)

Epoch 1/3
86/86 [==============================] - 2037s 23s/step - loss: 42.0224 - mean_absolute_error: 2.4562 - val_loss: 8.7039 - val_mean_absolute_error: 1.9832
Epoch 2/3
86/86 [==============================] - 2013s 23s/step - loss: 5.8001 - mean_absolute_error: 1.3261 - val_loss: 4.2971 - val_mean_absolute_error: 1.0469
Epoch 3/3
86/86 [==============================] - 1983s 23s/step - loss: 4.7873 - mean_absolute_error: 1.2029 - val_loss: 4.0539 - val_mean_absolute_error: 1.0580


In [ ]:
model.model.save('model23.h5')

In [ ]:
y_pred = model.prediction(X_test)

2668/2668 [==============================] - 482s 180ms/step


In [ ]:
print(y_pred)

[0.737354   0.3087861  0.75693005 ... 1.3162066  0.83206815 4.0509353 ]


In [ ]:
len(y_pred)

85372

In [ ]:
pred_df = pd.DataFrame(y_pred)
print(pred_df)
pred_df.to_csv('pred_23.csv')

              0
0      0.737354
1      0.308786
2      0.756930
3      1.225388
4      1.567861
...         ...
85367  0.911778
85368  0.750784
85369  1.316207
85370  0.832068
85371  4.050935

[85372 rows x 1 columns]


In [ ]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [ ]:
print("MSE validation %.3f" % mean_squared_error(y_true=y_test, y_pred=y_pred))
print("R2 score validation %.3f" % r2_score(y_true=y_test, y_pred=y_pred))
print("MAE validation %.3f" % mean_absolute_error(y_true=y_test, y_pred=y_pred))

MSE validation 5.805
R2 score validation 0.551
MAE validation 1.218


In [ ]:
c.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1)]          0           []                               
                                                                                              